In [6]:
import numpy as np
import pandas as pd
import requests

from bs4 import BeautifulSoup
from urllib.request import urlopen

import selenium
from selenium import webdriver
import time
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import ElementClickInterceptedException
from selenium.common.exceptions import StaleElementReferenceException
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import Select

import os
from PIL import Image

In [2]:
driver=webdriver.Chrome()
url='https://www.amazon.in/'
driver.get(url)
#take input from user
product=input('What are you looking for: ')

# find searchbox
searchbox=driver.find_element_by_xpath('//input[@id="twotabsearchtextbox"]')
# typing product in search box
searchbox.send_keys(product)
# press enter button
searchbox.send_keys(Keys.ENTER)


# creating empty list for storing data
brandname=[]
nameoftheproduct=[]
rating=[]
noOfrating=[]
price=[]
returnorexchange=[]
expectedDelivery=[]
availability=[]
otherdetails=[]
producturl=[]

#scrape data from 3 pages:
for i in range(3):
    np=driver.find_elements_by_xpath('//span[@class="a-size-medium a-color-base a-text-normal"]')
    for i in np:
        nameoftheproduct.append(i.text)
    rat=driver.find_elements_by_xpath('//span[@class="a-icon-alt"]')
    for j in rat:
        rating.append(j.text)
        
    ratc=driver.find_elements_by_xpath('//span[@class="a-size-base"]')
    for k in ratc:
        noOfrating.append(k.text)
        
    try:
            
        expd=driver.find_elements_by_xpath('//div[@class="a-row s-align-children-center"]')
        for l in expd:
            expectedDelivery.append(l.text)
    except:
        expectedDelivery.append('-')
        
    pricE=driver.find_elements_by_xpath('//span[@class="a-price-whole"]')
    for m in pricE:
        price.append(m.text)
        
    
amazon=pd.DataFrame()
amazon['Name of the product']=nameoftheproduct
amazon['Rating']=rating[:len(nameoftheproduct)]
amazon['No of Rating']=noOfrating[:len(nameoftheproduct)]
amazon['Price']=price[:len(nameoftheproduct)]

amazon


What are you looking for: guitar


,Name of the product,Rating,No of Rating,Price
0,Kadence Acoustica Series Semi Acoustic Ash Woo...,,597,"7,299"
1,Kadence Acoustica Series Semi Acoustic Ash Woo...,,"11,299","2,649"
2,Kadence Acoustica Series Semi Acoustic Ash Woo...,,259,"5,499"


Write a python program to access the search bar and search button on images.google.com and
scrape 100 images each for keywords ‘fruits’, ‘cars’ and ‘Machine Learning’.

In [3]:
import selenium
from selenium import webdriver
import time
import requests
import os
from PIL import Image
import io
import hashlib

# All in same directory

def fetch_image_urls(query:str, max_links_to_fetch:int, wd:webdriver, sleep_between_interactions:int=1):
    def scroll_to_end(wd):
        wd.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(sleep_between_interactions)        
    
    # build the google query
    search_url = "https://www.google.com/search?safe=off&site=&tbm=isch&source=hp&q={q}&oq={q}&gs_l=img"

    # load the page
    wd.get(search_url.format(q=query))

    image_urls = set()
    image_count = 0
    results_start = 0
    error_clicks = 0
    while (image_count < max_links_to_fetch) & (error_clicks < 30): # error clicks to stop when there are no more results to show by Google Images. You can tune the number
        scroll_to_end(wd)

        print('Starting search for Images')

        # get all image thumbnail results
        thumbnail_results = wd.find_elements_by_css_selector("img.Q4LuWd")
        number_results = len(thumbnail_results)
        
        print(f"Found: {number_results} search results. Extracting links from {results_start}:{number_results}")
        for img in thumbnail_results[results_start:max_links_to_fetch]:
            # try to click every thumbnail such that we can get the real image behind it
            print("Total Errors till now:", error_clicks)
            try:
                print('Trying to Click the Image')
                img.click()
                time.sleep(sleep_between_interactions)
                print('Image Click Successful!')
            except Exception:
                error_clicks = error_clicks + 1
                print('ERROR: Unable to Click the Image')
                if(results_start < number_results):
                    continue
                else:
                    break
            results_start = results_start + 1

            # extract image urls    
            print('Extracting of Image URLs')
            actual_images = wd.find_elements_by_css_selector('img.n3VNCb')
            for actual_image in actual_images:
                if actual_image.get_attribute('src') and 'http' in actual_image.get_attribute('src'):
                    image_urls.add(actual_image.get_attribute('src'))

            image_count = len(image_urls)

            print('Current Total Image Count:', image_count)

            if len(image_urls) >= max_links_to_fetch:
                print(f"Found: {len(image_urls)} image links, done!")
                break
            else:
                load_more_button = wd.find_element_by_css_selector(".mye4qd")
                if load_more_button:
                    wd.execute_script("document.querySelector('.mye4qd').click();")
       
        results_start = len(thumbnail_results)

    return image_urls

def persist_image(folder_path:str,file_name:str,url:str):
    try:
        image_content = requests.get(url).content

    except Exception as e:
        print(f"ERROR - Could not download {url} - {e}")

    try:
        image_file = io.BytesIO(image_content)
        image = Image.open(image_file).convert('RGB')
        folder_path = os.path.join(folder_path,file_name)
        if os.path.exists(folder_path):
            file_path = os.path.join(folder_path,hashlib.sha1(image_content).hexdigest()[:10] + '.jpg')
        else:
            os.mkdir(folder_path)
            file_path = os.path.join(folder_path,hashlib.sha1(image_content).hexdigest()[:10] + '.jpg')
        with open(file_path, 'wb') as f:
            image.save(f, "JPEG", quality=85)
        print(f"SUCCESS - saved {url} - as {file_path}")
    except Exception as e:
        print(f"ERROR - Could not save {url} - {e}")
        

In [4]:
wd = webdriver.Chrome()
wd.get('https://google.com')
search_box = wd.find_element_by_css_selector('input.gLFyf')
searchelement='fruit'
search_box.send_keys(searchelement)
links = fetch_image_urls(searchelement,100,wd)
images_path = '/home/nik/Desktop/Google images'
for i in links:
    persist_image(images_path,searchelement,i)
    
wd.quit()

Starting search for Images
Found: 100 search results. Extracting links from 0:100
Total Errors till now: 0
Trying to Click the Image
Image Click Successful!
Extracting of Image URLs
Current Total Image Count: 0
Total Errors till now: 0
Trying to Click the Image
Image Click Successful!
Extracting of Image URLs
Current Total Image Count: 1
Total Errors till now: 0
Trying to Click the Image
Image Click Successful!
Extracting of Image URLs
Current Total Image Count: 2
Total Errors till now: 0
Trying to Click the Image
Image Click Successful!
Extracting of Image URLs
Current Total Image Count: 3
Total Errors till now: 0
Trying to Click the Image
Image Click Successful!
Extracting of Image URLs
Current Total Image Count: 4
Total Errors till now: 0
Trying to Click the Image
Image Click Successful!
Extracting of Image URLs
Current Total Image Count: 5
Total Errors till now: 0
Trying to Click the Image
Image Click Successful!
Extracting of Image URLs
Current Total Image Count: 6
Total Errors ti

Image Click Successful!
Extracting of Image URLs
Current Total Image Count: 90
Total Errors till now: 0
Trying to Click the Image
Image Click Successful!
Extracting of Image URLs
Current Total Image Count: 91
Total Errors till now: 0
Trying to Click the Image
Image Click Successful!
Extracting of Image URLs
Current Total Image Count: 93
Total Errors till now: 0
Trying to Click the Image
Image Click Successful!
Extracting of Image URLs
Current Total Image Count: 95
Total Errors till now: 0
Trying to Click the Image
Image Click Successful!
Extracting of Image URLs
Current Total Image Count: 97
Total Errors till now: 0
Trying to Click the Image
Image Click Successful!
Extracting of Image URLs
Current Total Image Count: 99
Total Errors till now: 0
Trying to Click the Image
Image Click Successful!
Extracting of Image URLs
Current Total Image Count: 100
Found: 100 image links, done!
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcRAFt0xHiff3fl4uAzBm9kkAa36Rnq2-i1lHQ&usq

SUCCESS - saved https://images.unsplash.com/photo-1601004890684-d8cbf643f5f2?ixlib=rb-1.2.1&ixid=MnwxMjA3fDB8MHxleHBsb3JlLWZlZWR8Mnx8fGVufDB8fHx8&w=1000&q=80 - as /home/nik/Desktop/Google images/fruit/40536a3bd9.jpg
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcS-gxDyIQ1tOuvQjXk2SqTj9aMD6dy5C0-BJA&usqp=CAU - as /home/nik/Desktop/Google images/fruit/2411f5c712.jpg
SUCCESS - saved https://cdn.britannica.com/86/75886-050-B481C97E/Apricots.jpg - as /home/nik/Desktop/Google images/fruit/034236c1f1.jpg
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcT4fqXfgTypAMphAwIjfmK5Gvzfzj-5KQVKRA&usqp=CAU - as /home/nik/Desktop/Google images/fruit/72e5f31be5.jpg
SUCCESS - saved https://www.byrdie.com/thmb/yLP5JCjmHF35-1XmKMQ9DffsZIU=/2119x1414/filters:no_upscale():max_bytes(150000):strip_icc()/GettyImages-964480900-2b128ee35ecc4758a08d28e75e3b9a96.jpg - as /home/nik/Desktop/Google images/fruit/3299da9f57.jpg
SUCCESS - saved https://encrypted-tbn0.gstatic.co

/home/nik/anaconda3/lib/python3.8/site-packages/PIL/Image.py:951: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


SUCCESS - saved https://www.dole.com/-/media/project/dole/produce-images/headers/dole-produce-fruit-medley.png?h=540&w=720&rev=1416123f2d094cd1b7494365948214be&hash=09AD8D65F7B13901453125B22EE62C00 - as /home/nik/Desktop/Google images/fruit/0adba64d1e.jpg
SUCCESS - saved https://img.freepik.com/free-vector/flat-design-fruit-collection_23-2148933782.jpg?size=338&ext=jpg - as /home/nik/Desktop/Google images/fruit/b17ed4e5fa.jpg
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcQgLKKTKfndP1PwfcFqlKTgCSTXOkgpsbI2GQ&usqp=CAU - as /home/nik/Desktop/Google images/fruit/a28789335a.jpg
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcRzvz77EAIva3gKMyKNlY341hRpSZ5rtpuNRA&usqp=CAU - as /home/nik/Desktop/Google images/fruit/bafd158f20.jpg
SUCCESS - saved https://res.cloudinary.com/grohealth/image/upload/$wpsize_!_cld_full!,w_2100,h_1427,c_scale/v1588088840/iStock-467652436.jpg - as /home/nik/Desktop/Google images/fruit/2cd8f66978.jpg
SUCCESS - saved https:/

In [5]:
wd = webdriver.Chrome()
wd.get('https://google.com')
search_box = wd.find_element_by_css_selector('input.gLFyf')
searchelement='car'
search_box.send_keys(searchelement)
links = fetch_image_urls(searchelement,100,wd)
images_path = '/home/nik/Desktop/Google images'
for i in links:
    persist_image(images_path,searchelement,i)
    
wd.quit()

Starting search for Images
Found: 100 search results. Extracting links from 0:100
Total Errors till now: 0
Trying to Click the Image
Image Click Successful!
Extracting of Image URLs
Current Total Image Count: 0
Total Errors till now: 0
Trying to Click the Image
Image Click Successful!
Extracting of Image URLs
Current Total Image Count: 1
Total Errors till now: 0
Trying to Click the Image
Image Click Successful!
Extracting of Image URLs
Current Total Image Count: 2
Total Errors till now: 0
Trying to Click the Image
Image Click Successful!
Extracting of Image URLs
Current Total Image Count: 3
Total Errors till now: 0
Trying to Click the Image
Image Click Successful!
Extracting of Image URLs
Current Total Image Count: 4
Total Errors till now: 0
Trying to Click the Image
Image Click Successful!
Extracting of Image URLs
Current Total Image Count: 5
Total Errors till now: 0
Trying to Click the Image
Image Click Successful!
Extracting of Image URLs
Current Total Image Count: 6
Total Errors ti

SUCCESS - saved https://media-cldnry.s-nbcnews.com/image/upload/newscms/2018_12/2371046/180320-flying-car-aeromobil50-se-143p.jpg - as /home/nik/Desktop/Google images/car/ea0a8f0d53.jpg
SUCCESS - saved https://static.dw.com/image/51817152_6.jpg - as /home/nik/Desktop/Google images/car/0dd4dc5279.jpg
SUCCESS - saved https://i.ytimg.com/vi/dip_8dmrcaU/maxresdefault.jpg - as /home/nik/Desktop/Google images/car/c024fc29a6.jpg
SUCCESS - saved https://image.cnbcfm.com/api/v1/image/106845268-1614203269431-AppleCar_YT_Thumbnail_Option.png?v=1614203346 - as /home/nik/Desktop/Google images/car/84dbeb38af.jpg
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcSu5mwXZ9iCFwuuXxVXai-VSUBnSrfA5FSqTg&usqp=CAU - as /home/nik/Desktop/Google images/car/6375dfdb03.jpg
SUCCESS - saved https://hips.hearstapps.com/amv-prod-gp.s3.amazonaws.com/gearpatrol/wp-content/uploads/2019/10/Buy-a-Kia-Telluride-Instead-gear-patrol-slide-1.jpg?crop=0.620xw:0.919xh;0.293xw,0.0813xh&resize=640:* - as /ho

KeyboardInterrupt: 

In [6]:
wd = webdriver.Chrome()
wd.get('https://google.com')
search_box = wd.find_element_by_css_selector('input.gLFyf')
searchelement='machine learning'
search_box.send_keys(searchelement)
links = fetch_image_urls(searchelement,100,wd)
images_path = '/home/nik/Desktop/Google images'
for i in links:
    persist_image(images_path,searchelement,i)
    
wd.quit()

Starting search for Images
Found: 100 search results. Extracting links from 0:100
Total Errors till now: 0
Trying to Click the Image
Image Click Successful!
Extracting of Image URLs
Current Total Image Count: 1
Total Errors till now: 0
Trying to Click the Image
Image Click Successful!
Extracting of Image URLs
Current Total Image Count: 2
Total Errors till now: 0
Trying to Click the Image
Image Click Successful!
Extracting of Image URLs
Current Total Image Count: 3
Total Errors till now: 0
Trying to Click the Image
Image Click Successful!
Extracting of Image URLs
Current Total Image Count: 4
Total Errors till now: 0
Trying to Click the Image
Image Click Successful!
Extracting of Image URLs
Current Total Image Count: 4
Total Errors till now: 0
Trying to Click the Image
Image Click Successful!
Extracting of Image URLs
Current Total Image Count: 4
Total Errors till now: 0
Trying to Click the Image
Image Click Successful!
Extracting of Image URLs
Current Total Image Count: 4
Total Errors ti

Image Click Successful!
Extracting of Image URLs
Current Total Image Count: 72
Total Errors till now: 0
Trying to Click the Image
Image Click Successful!
Extracting of Image URLs
Current Total Image Count: 73
Total Errors till now: 0
Trying to Click the Image
Image Click Successful!
Extracting of Image URLs
Current Total Image Count: 75
Total Errors till now: 0
Trying to Click the Image
Image Click Successful!
Extracting of Image URLs
Current Total Image Count: 76
Total Errors till now: 0
Trying to Click the Image
Image Click Successful!
Extracting of Image URLs
Current Total Image Count: 78
Total Errors till now: 0
Trying to Click the Image
Image Click Successful!
Extracting of Image URLs
Current Total Image Count: 79
Total Errors till now: 0
Trying to Click the Image
Image Click Successful!
Extracting of Image URLs
Current Total Image Count: 80
Total Errors till now: 0
Trying to Click the Image
Image Click Successful!
Extracting of Image URLs
Current Total Image Count: 81
Total Error

/home/nik/anaconda3/lib/python3.8/site-packages/PIL/Image.py:951: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


SUCCESS - saved https://mobidev.biz/wp-content/uploads/2020/09/machine-learning-in-demand-forecasting.png - as /home/nik/Desktop/Google images/machine learning/af7b725278.jpg
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcQi91_Jdrzz4j1yii7sbMoBxoIisF4XG4i8Pw&usqp=CAU - as /home/nik/Desktop/Google images/machine learning/12c68fdd2c.jpg
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcSRkU62g7Rhz3aNupBoOc6pizY-19hH9-MJKA&usqp=CAU - as /home/nik/Desktop/Google images/machine learning/13cae6fbb2.jpg
SUCCESS - saved https://www.sas.com/en_in/insights/analytics/machine-learning/_jcr_content/par/styledcontainer_31e3/image.img.jpg/1626474997609.jpg - as /home/nik/Desktop/Google images/machine learning/adb77f54cf.jpg
SUCCESS - saved https://miro.medium.com/max/1400/0*QYxNNYh6W9jO1b_-.png - as /home/nik/Desktop/Google images/machine learning/5fef73c0d8.jpg
SUCCESS - saved https://appen.com/wp-content/uploads/2018/09/machine-learning-wiki.jpg - as /home

SUCCESS - saved https://data-flair.training/blogs/wp-content/uploads/sites/2/2017/07/what-is-machine-learning-1200x675.jpg - as /home/nik/Desktop/Google images/machine learning/3af442ab3f.jpg
SUCCESS - saved https://serokell.io/files/zx/zxwju3ha.Machine-learning-vs-deep-learning.jpg - as /home/nik/Desktop/Google images/machine learning/59cbf94e42.jpg
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcQon8ctAdIzbccLnnxKfNCwtGzpylYQP3D0Mw&usqp=CAU - as /home/nik/Desktop/Google images/machine learning/b4a596f389.jpg
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcSfYan72d3-aB9N1urTRzUvlPgiuo9p8qk0qw&usqp=CAU - as /home/nik/Desktop/Google images/machine learning/63ae46ca20.jpg
SUCCESS - saved https://www.simplilearn.com/ice9/free_resources_article_thumb/Deep-Learning-vs-Machine-Learning.jpg - as /home/nik/Desktop/Google images/machine learning/de1b1079eb.jpg
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcQ9ZhBDEGVp-DOA_hJ917nU

/home/nik/anaconda3/lib/python3.8/site-packages/PIL/Image.py:2942: UserWarning: image file could not be identified because WEBP support not installed
  warnings.warn(message)


ERROR - Could not save https://www.botreetechnologies.com/blog/wp-content/uploads/2020/07/al-vs-machine-learning-vs-deep-learning-difference.png - cannot identify image file <_io.BytesIO object at 0x7eff9490e040>
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcTVTbGRhJ57v7V1QD7ryIDXwcmP-QqzvnH8NQ&usqp=CAU - as /home/nik/Desktop/Google images/machine learning/6461b300cb.jpg
SUCCESS - saved http://www.metal-am.com/wp-content/uploads/sites/4/2020/12/fig-01-copy-1024x630.jpg - as /home/nik/Desktop/Google images/machine learning/d81be17808.jpg
SUCCESS - saved https://www.oracle.com/a/ocom/img/rc24-machine-learning.jpg - as /home/nik/Desktop/Google images/machine learning/1dab0a134d.jpg
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcSx7Od_I1PpCMQCLAWuR5ZPoSrSbkbUWa0Hig&usqp=CAU - as /home/nik/Desktop/Google images/machine learning/f3a2421b2d.jpg
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcS8B-h-XhDxP5a7Qd-HPdBoHiB1hbafQU_

Write a python program to search for a smartphone(e.g.: Oneplus Nord, pixel 4A, etc.) on
www.flipkart.com and scrape following details for all the search results displayed on 1 st page.
Details to be scraped: “Brand Name”, “Smartphone name”, “Colour”, “RAM”,
“Storage(ROM)”, “Primary Camera”, “Secondary Camera”, “Display Size”, “Display
Resolution”, “Processor”, “Processor Cores”, “Battery Capacity”, “Price”, “Product URL”.
Incase if any of the details is missing then replace it by “- “. Save your results in a dataframe
and CSV.

In [4]:
flipkartdriver=webdriver.Chrome()
url='http://www.flipkart.com/'
flipkartdriver.get(url)

# remove signin page
signin=flipkartdriver.find_element_by_xpath('//html/body/div[2]/div/div/button')
signin.click()

# searchbar
searchBar=flipkartdriver.find_element_by_xpath('//*[@id="container"]/div/div[1]/div[1]/div[2]/div[2]/form/div/div/input')
searchBar.send_keys('samsung phone')
searchBar.send_keys(Keys.ENTER)

In [5]:
urls=[]
furl=flipkartdriver.find_elements_by_xpath('//a[@class="_1fQZEK"]')
for ur in furl:
    urls.append(ur.get_attribute('href'))
    
smartphoneName=[]
colour[]
ram=[]
rom=[]
primarycamera=[]
secondarycamera=[]
displaysize=[]
displayresolution=[]
processors=[]
processorcore=[]
batterycapacity=[]
price=[]
producturl=[]

    
for urlS in urls:
    flipkartdriver.get(urlS)
    smpname=flipkartdriver.find_element_by_xpath('//span[@class="B_NuCI"]')
    print(smartphoneName.append(smpname.text))
     
    
    highlights=flipkartdriver.find_element_by_xpath('//div[@class="_2418kt"]')
    highlight.append(highlights.text)
    
print(smartphoneName)
print(highlight)
    
#     more=flipkartdriver.find_element_by_xpath('//[@class="_2KpZ6l _1FH0tX"]')
#     more.click()
# print(smartphoneName)
# print(price)
# # print(highlight)
    
    



# smartphoneName
    
# prices=[]
# for price in flipkartdriver.find_elements_by_xpath('//div[@class="_30jeq3 _1_WHN1"]'):
#     prices.append(price.text)

    
# alldetails=[]
# for alldetail in flipkartdriver.find_elements_by_xpath('//ul[@class="_1xgFaf"]'):
#     alldetails.append(alldetail.text)
# count=1
# for i in alldetails:
#     print(count)
#     print(i)
#     count+=1

None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
['SAMSUNG GALAXY M31S (Mirage Black, 128 GB)  (8 GB RAM)', 'SAMSUNG Galaxy F22 (Denim Blue, 64 GB)  (4 GB RAM)', 'SAMSUNG Galaxy F22 (Denim Blue, 128 GB)  (6 GB RAM)', 'SAMSUNG Galaxy F22 (Denim Black, 64 GB)  (4 GB RAM)', 'SAMSUNG Galaxy F12 (Sky Blue, 128 GB)  (4 GB RAM)', 'SAMSUNG Galaxy F12 (Celestial Black, 128 GB)  (4 GB RAM)', 'SAMSUNG Guru 1200  (White)', 'SAMSUNG Galaxy M42 5G (Prism Dot Gray, 128 GB)  (6 GB RAM)', 'SAMSUNG Guru FM Plus SM-B110E/D  (Black)', 'SAMSUNG Guru Music 2  (Blue)', 'SAMSUNG Galaxy A31 (Prism Crush Blue, 128 GB)  (6 GB RAM)', 'SAMSUNG Metro 313 Dual Sim  (Gold)', 'SAMSUNG Guru FM Plus  (Gold)', 'SAMSUNG M31 (Ocean Blue, 128 GB)  (6 GB RAM)', 'SAMSUNG Galaxy M31 (Space Black, 128 GB)  (8 GB RAM)', 'SAMSUNG Guru FM Plus SM-B110E/D  (Dark Blue)', 'SAMSUNG Galaxy F02s (Diamond Blue, 64 GB)  (4 GB RAM)', 'SAMSUNG Galaxy M01 (Black, 32 GB)  

In [30]:
hostelworld=webdriver.Chrome()
hostelworld.get('https://www.hostelworld.com/hostels')
hostelworld.maximize_window()
hostelworld.implicitly_wait(20)

search=hostelworld.find_element_by_xpath('//input[@name="search_keywords"]')
search.send_keys('London')
suggestions=hostelworld.find_element_by_xpath('//li[@class="hover"]')
suggestions.click()

submit=hostelworld.find_element_by_xpath('//button[@type="submit"]')
submit.click()


In [31]:
hostelname=[]
distancefromcitycenter=[]
rating=[]
totalreview=[]
overallreview=[]
prices=[]
urls=[]
description=[]

for count in range(3):
    
    hostelnamE=hostelworld.find_elements_by_xpath('//h2[@class="title title-6"]')
    for hostelName in hostelnamE:
        hostelname.append(hostelName.text)
        
    distancefromcitycenteR=hostelworld.find_elements_by_xpath('//span[@class="description"]')
    for distanceFromcityCenter in distancefromcitycenteR:
        distancefromcitycenter.append(distanceFromcityCenter.text)
        
    try:
        ratinG=hostelworld.find_elements_by_xpath('//div[@class="score orange big"]')
    except:
        rating.append('-')
    for raTing in ratinG:
        rating.append(raTing.text)
        
    totalrevieW=hostelworld.find_elements_by_xpath('//div[@class="reviews"]')
    for totalReview in totalrevieW:
         totalreview.append(totalReview.text)
            
    overallrevieW=hostelworld.find_elements_by_xpath('//div[@class="keyword"]/span')
    for overallReview in overallrevieW:
        overallreview.append(overallReview.text)
        
    pricE=hostelworld.find_elements_by_xpath('//div[@class="price-col"]')
    for price in pricE:
        prices.append(price.text.replace('\n',' '))
        
    
    allurls=hostelworld.find_elements_by_xpath('//h2[@class="title title-6"]/a')
    for url in allurls:
        urls.append(url.get_attribute('href'))
        
    try:
              
        nextpage=hostelworld.find_element_by_xpath('//i[@class="core-icon icon-core-chevron-right"]')
        nextpage.click()
        
    except:
        
        pass

In [32]:
private=prices[0::2]
dorms=prices[1::2]
for u in urls:
    hostelworld.get(u)
    hostelworld.execute_script("window.scrollTo(0, 100);")
    
    try:
        
        content=hostelworld.find_element_by_xpath('//div[@class="content collapse-content"]')
        description.append(content.text.replace('\n',' '))
        
    except:
        content=hostelworld.find_element_by_xpath('//div[@class="content"]')
        description.append(content.text.replace('\n',' '))
    

In [38]:
    
hostel=pd.DataFrame()
hostel['Name']=hostelname[:53]
hostel['Distance from city center']=distancefromcitycenter[:53]
hostel['Rating']=rating[:53]
hostel['Total Review']=totalreview[:53]
hostel['Overall Review']=overallreview[:53]
hostel['Private from price']=private[:53]
hostel['Dorms from price']=dorms[:53]
hostel['Description']=description[:53]

hostel

,Name,Distance from city center,Rating,Total Review,Overall Review,Private from price,Dorms from price,Description
0,St Christopher's Village,Hostel - 1.8km from city centre,8.8,10819 Total Reviews,Superb,Privates From Rs3752,Dorms From Rs1457,COVID 19 Policy Update. In response to Coronav...
1,Generator London,Hostel - 3km from city centre,7.6,6733 Total Reviews,Superb,Privates From Rs8425,Dorms From Rs1978,Generator London is a design hotel-hostel loca...
2,Safestay London Kensington Holland Park,Hostel - 5.9km from city centre,9.8,1074 Total Reviews,Fabulous,No Privates Available,Dorms From Rs1105,Safestay Holland Park Safestay Holland Park i...
3,"PubLove @ The Crown, Battersea",Hostel - 4.7km from city centre,6.8,207 Total Reviews,Fabulous,No Privates Available,Dorms From Rs1706,Do London like a local. Stay just outside the ...
4,247london Hostel and Private Rooms,Hostel - 8.2km from city centre,8.9,443 Total Reviews,Very Good,Privates From Rs5219,No Dorms Available,247london Hostel and Private Rooms is a home a...
5,"PubLove @ The Steam Engine, Waterloo",Hostel - 0.5km from city centre,7.7,186 Total Reviews,Superb,Privates From Rs13954,Dorms From Rs1671,The heart & soul of London backpacking Pull up...
6,The London Home Hostel,Hostel - 3.6km from city centre,9.3,248 Total Reviews,Good,Privates From Rs4093,Dorms From Rs1617,If you are looking to meet and socialise with ...
7,Wombat's The City Hostel London,Hostel - 3.6km from city centre,8.1,13129 Total Reviews,No Rating,Privates From Rs9221,Dorms From Rs2196,Wombat's The City Hostel London is definitely ...
8,Safestay London Elephant & Castle,Hostel - 1.7km from city centre,6.9,3945 Total Reviews,Fabulous,Privates From Rs9210,Dorms From Rs1074,Safestay at Elephant & Castle is ideal if you ...
9,"PubLove @ The Green Man, Paddington",Hostel - 4.3km from city centre,9.5,391 Total Reviews,Very Good,Privates From Rs8665,Dorms From Rs1569,Where it all began - PubLove was born here in ...


In [39]:
# save data in csv file format.
hostel.to_csv('hostelworld.csv')

In [7]:
forbes=webdriver.Chrome()
forbes.get('https://www.forbes.com/billionaires/')

In [8]:
personName=[]
fperson=forbes.find_elements_by_xpath('//div[@class="personName"]')
for f in fperson:
    personName.append(f.text)
    
networth=[]
fnetworth=forbes.find_elements_by_xpath('//div[@class="netWorth"]')
for fn in fnetworth:
    networth.append(fn.text)
    
age=[]
fage=forbes.find_elements_by_xpath('//div[@class="age"]')
for fa in fage:
    age.append(fa.text)
    
citizenship=[]
fcititzenship=forbes.find_elements_by_xpath('//div[@class="countryOfCitizenship"]')
for fc in fcititzenship:
    citizenship.append(fc.text)

source=[]
fsource=forbes.find_elements_by_xpath('//div[@class="source"]')
for fs in fsource:
    source.append(fs.text)

industry=[]
findustry=forbes.find_elements_by_xpath('//div[@class="category"]')
for ff in findustry:
    industry.append(ff.text)

forebslist=pd.DataFrame()
forebslist['Name']=personName
forebslist['Net worth']=networth
forebslist['Age']=age
forebslist['Citizenship']=citizenship
forebslist['Source']=source
forebslist['Industry']=industry
forebslist.index=np.arange(1,len(forebslist)+1)

forebslist

,Name,Net worth,Age,Citizenship,Source,Industry
1,Jeff Bezos,$177 B,,United States,Amazon,
2,Elon Musk,$151 B,,United States,"Tesla, SpaceX",
3,Bernard Arnault & family,$150 B,,France,LVMH,
4,Bill Gates,$124 B,,United States,Microsoft,
5,Mark Zuckerberg,$97 B,,United States,Facebook,
...,...,...,...,...,...,...
196,Harry Triguboff,$11.2 B,,Australia,real estate,
197,Leonid Fedun & family,$11.1 B,,Russia,oil,
198,Eyal Ofer,$11.1 B,,Israel,"real estate, shipping",
199,Evan Spiegel,$11.1 B,,United States,Snapchat,


In [9]:
# store data in csv format
forebslist.to_csv('forebesbillionaries.csv')